<a href="https://colab.research.google.com/github/naharyana/Thesis/blob/master/Copy_of_Split_Spam_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [ ]:
import gensim

In [ ]:
import pickle

In [ ]:
fasttext_model = gensim.models.FastText.load('/content/drive/My Drive/Colab Notebooks/Text_gen/Juli/model/fasttext_trained_model_s')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


OSError: ignored

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/Text_gen/Juli/model/fasttext_trained_model_s_juli.pickle", "rb") as fp:   # Unpickling
    ft_weights = pickle.load(fp)

In [ ]:
ft_weights # see whole vector array

In [ ]:
fasttext_model_wv = fasttext_model.wv

In [ ]:
fasttext_model_wv['good']

In [ ]:
fasttext_model_wv.most_similar('good')

In [ ]:
def word2idx(word):
    try:
        return fasttext_model_wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
    
def idx2word(idx):
    return fasttext_model_wv.index2word[idx]

In [ ]:
fasttext_model_wv.index2word[89]


In [ ]:
vocab_size, embedding_layer_output_size = ft_weights.shape
print(vocab_size, embedding_layer_output_size)

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
 # load
in_filename = '/content/drive/My Drive/Colab Notebooks/Text_gen/data/spam_seq_sel_done.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size_y = len(tokenizer.word_index) + 1

In [ ]:
sequences[0]

In [ ]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size_y)
seq_length = X.shape[1]

In [ ]:
seq_length

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


In [ ]:
import nltk
word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in lines]

In [ ]:
# define model
#model = Sequential()
#model.add(Embedding(input_dim=561, # vocab size
#                    output_dim=embedding_layer_output_size, # 100
#                    weights=[ft_weights],
#                    input_length=200, # 200
#                    trainable=False))
#vocab_size, 50, input_lenth=seq_length
#model.add(Dropout(0.2))
#model.add(Conv1D(64, 5, activation='relu'))
#model.add(MaxPooling1D(pool_size=4))
#model.add(LSTM(100))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(vocab_size, activation='softmax'))
#print(model.summary())

In [ ]:
# define model ok
model = Sequential()
model.add(Embedding(input_dim=vocab_size_y, # 34867
                    output_dim=embedding_layer_output_size, # 100
                    input_length=50)) # 200
#vocab_size, 50, input_lenth=seq_length
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
#model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size_y, activation='softmax'))
print(model.summary())

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit model
model.fit(X, y, validation_split=0.33, batch_size=128, epochs=100)

In [ ]:
# save the model to file
model.save('/content/drive/My Drive/Colab Notebooks/Text_gen/Juli/model/model_spam_cnn_juli_dropout.h5')

In [ ]:
# save the tokenizer
dump(tokenizer, open('/content/drive/My Drive/Colab Notebooks/Text_gen/Juli/model/tokenizer_spam_cnn_juli_dropout.pkl', 'wb'))

In [ ]:
import pandas as pd
x_t = pd.DataFrame(X_test)
y_t = pd.DataFrame(y_test)

In [ ]:
import numpy as np
#encoded_argmax  = np.argmax(array(X_test), axis=1)
text_test = tokenizer.sequences_to_texts(X_test)

In [ ]:
with open('test_file_juli.txt', 'w') as f:
    for item in text_test:
        f.write("%s\n" % item)